In [ ]:
import numpy as np

from ._base import _fit_liblinear, BaseSVC, BaseLibSVM
from ..base import BaseEstimator, RegressorMixin, OutlierMixin
from ..linear_model._base import LinearClassifierMixin, SparseCoefMixin, \
    LinearModel
from ..utils.validation import _num_samples
from ..utils.validation import _deprecate_positional_args
from ..utils.multiclass import check_classification_targets
from ..utils.deprecation import deprecated


class LinearSVC(BaseEstimator, LinearClassifierMixin,
                SparseCoefMixin):
    
    def __init__(self, penalty='l2', loss='squared_hinge', *, dual=True,
                 tol=1e-4, C=1.0, multi_class='ovr', fit_intercept=True,
                 intercept_scaling=1, class_weight=None, verbose=0,
                 random_state=None, max_iter=1000):
        self.dual = dual
        self.tol = tol
        self.C = C
        self.multi_class = multi_class
        self.fit_intercept = fit_intercept
        self.intercept_scaling = intercept_scaling
        self.class_weight = class_weight
        self.verbose = verbose
        self.random_state = random_state
        self.max_iter = max_iter
        self.penalty = penalty
        self.loss = loss

    def fit(self, X, y, sample_weight=None):
        if self.C < 0:
            raise ValueError("Penalty term must be positive; got (C=%r)"
                             % self.C)

        X, y = self._validate_data(X, y, accept_sparse='csr',
                                   dtype=np.float64, order="C",
                                   accept_large_sparse=False)
        check_classification_targets(y)
        self.classes_ = np.unique(y)

        self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
            X, y, self.C, self.fit_intercept, self.intercept_scaling,
            self.class_weight, self.penalty, self.dual, self.verbose,
            self.max_iter, self.tol, self.random_state, self.multi_class,
            self.loss, sample_weight=sample_weight)

        if self.multi_class == "crammer_singer" and len(self.classes_) == 2:
            self.coef_ = (self.coef_[1] - self.coef_[0]).reshape(1, -1)
            if self.fit_intercept:
                intercept = self.intercept_[1] - self.intercept_[0]
                self.intercept_ = np.array([intercept])

        return self


class LinearSVR(RegressorMixin, LinearModel):
    
    @_deprecate_positional_args
    def __init__(self, *, epsilon=0.0, tol=1e-4, C=1.0,
                 loss='epsilon_insensitive', fit_intercept=True,
                 intercept_scaling=1., dual=True, verbose=0,
                 random_state=None, max_iter=1000):
        self.tol = tol
        self.C = C
        self.epsilon = epsilon
        self.fit_intercept = fit_intercept
        self.intercept_scaling = intercept_scaling
        self.verbose = verbose
        self.random_state = random_state
        self.max_iter = max_iter
        self.dual = dual
        self.loss = loss

    def fit(self, X, y, sample_weight=None):
        if self.C < 0:
            raise ValueError("Penalty term must be positive; got (C=%r)"
                             % self.C)

        X, y = self._validate_data(X, y, accept_sparse='csr',
                                   dtype=np.float64, order="C",
                                   accept_large_sparse=False)
        penalty = 'l2'  # SVR only accepts l2 penalty
        self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
            X, y, self.C, self.fit_intercept, self.intercept_scaling,
            None, penalty, self.dual, self.verbose,
            self.max_iter, self.tol, self.random_state, loss=self.loss,
            epsilon=self.epsilon, sample_weight=sample_weight)
        self.coef_ = self.coef_.ravel()

        return self


class SVC(BaseSVC):
    
    _impl = 'c_svc'

    @_deprecate_positional_args
    def __init__(self, *, C=1.0, kernel='rbf', degree=3, gamma='scale',
                 coef0=0.0, shrinking=True, probability=False,
                 tol=1e-3, cache_size=200, class_weight=None,
                 verbose=False, max_iter=-1, decision_function_shape='ovr',
                 break_ties=False,
                 random_state=None):

        super().__init__(
            kernel=kernel, degree=degree, gamma=gamma,
            coef0=coef0, tol=tol, C=C, nu=0., shrinking=shrinking,
            probability=probability, cache_size=cache_size,
            class_weight=class_weight, verbose=verbose, max_iter=max_iter,
            decision_function_shape=decision_function_shape,
            break_ties=break_ties,
            random_state=random_state)


class NuSVC(BaseSVC):
    

    _impl = 'nu_svc'

    @_deprecate_positional_args
    def __init__(self, *, nu=0.5, kernel='rbf', degree=3, gamma='scale',
                 coef0=0.0, shrinking=True, probability=False, tol=1e-3,
                 cache_size=200, class_weight=None, verbose=False, max_iter=-1,
                 decision_function_shape='ovr', break_ties=False,
                 random_state=None):

        super().__init__(
            kernel=kernel, degree=degree, gamma=gamma,
            coef0=coef0, tol=tol, C=0., nu=nu, shrinking=shrinking,
            probability=probability, cache_size=cache_size,
            class_weight=class_weight, verbose=verbose, max_iter=max_iter,
            decision_function_shape=decision_function_shape,
            break_ties=break_ties,
            random_state=random_state)

    def _more_tags(self):
        return {
            '_xfail_checks': {
                'check_methods_subset_invariance':
                'fails for the decision_function method',
                'check_class_weight_classifiers': 'class_weight is ignored.'
            }
        }


class SVR(RegressorMixin, BaseLibSVM):
    
    _impl = 'epsilon_svr'

    @_deprecate_positional_args
    def __init__(self, *, kernel='rbf', degree=3, gamma='scale',
                 coef0=0.0, tol=1e-3, C=1.0, epsilon=0.1, shrinking=True,
                 cache_size=200, verbose=False, max_iter=-1):

        super().__init__(
            kernel=kernel, degree=degree, gamma=gamma,
            coef0=coef0, tol=tol, C=C, nu=0., epsilon=epsilon, verbose=verbose,
            shrinking=shrinking, probability=False, cache_size=cache_size,
            class_weight=None, max_iter=max_iter, random_state=None)

    # mypy error: Decorated property not supported
    @deprecated(  # type: ignore
        "The probA_ attribute is deprecated in version 0.23 and will be "
        "removed in version 0.25.")
    @property
    def probA_(self):
        return self._probA

    # mypy error: Decorated property not supported
    @deprecated(  # type: ignore
        "The probB_ attribute is deprecated in version 0.23 and will be "
        "removed in version 0.25.")
    @property
    def probB_(self):
        return self._probB


class NuSVR(RegressorMixin, BaseLibSVM):
    
    _impl = 'nu_svr'

    @_deprecate_positional_args
    def __init__(self, *, nu=0.5, C=1.0, kernel='rbf', degree=3,
                 gamma='scale', coef0=0.0, shrinking=True,
                 tol=1e-3, cache_size=200, verbose=False, max_iter=-1):

        super().__init__(
            kernel=kernel, degree=degree, gamma=gamma, coef0=coef0,
            tol=tol, C=C, nu=nu, epsilon=0., shrinking=shrinking,
            probability=False, cache_size=cache_size, class_weight=None,
            verbose=verbose, max_iter=max_iter, random_state=None)


class OneClassSVM(OutlierMixin, BaseLibSVM):

    _impl = 'one_class'

    @_deprecate_positional_args
    def __init__(self, *, kernel='rbf', degree=3, gamma='scale',
                 coef0=0.0, tol=1e-3, nu=0.5, shrinking=True, cache_size=200,
                 verbose=False, max_iter=-1):

        super().__init__(
            kernel, degree, gamma, coef0, tol, 0., nu, 0.,
            shrinking, False, cache_size, None, verbose, max_iter,
            random_state=None)

    def fit(self, X, y=None, sample_weight=None, **params):
        super().fit(X, np.ones(_num_samples(X)),
                    sample_weight=sample_weight, **params)
        self.offset_ = -self._intercept_
        return self

    def decision_function(self, X):
        dec = self._decision_function(X).ravel()
        return dec

    def score_samples(self, X):
        return self.decision_function(X) + self.offset_

    def predict(self, X):
        y = super().predict(X)
        return np.asarray(y, dtype=np.intp)

    # mypy error: Decorated property not supported
    @deprecated(  # type: ignore
        "The probA_ attribute is deprecated in version 0.23 and will be "
        "removed in version 0.25.")
    @property
    def probA_(self):
        return self._probA

    # mypy error: Decorated property not supported
    @deprecated(  # type: ignore
        "The probB_ attribute is deprecated in version 0.23 and will be "
        "removed in version 0.25.")
    @property
    def probB_(self):
        return self._probB